![cyber_photo](cyber_photo.jpg)

Cyber threats are a growing concern for organizations worldwide. These threats take many forms, including malware, phishing, and denial-of-service (DOS) attacks, compromising sensitive information and disrupting operations. The increasing sophistication and frequency of these attacks make it imperative for organizations to adopt advanced security measures. Traditional threat detection methods often fall short due to their inability to adapt to new and evolving threats. This is where deep learning models come into play.

Deep learning models can analyze vast amounts of data and identify patterns that may not be immediately obvious to human analysts. By leveraging these models, organizations can proactively detect and mitigate cyber threats, safeguarding their sensitive information and ensuring operational continuity.

As a cybersecurity analyst, you identify and mitigate these threats. In this project, you will design and implement a deep learning model to detect cyber threats. The BETH dataset simulates real-world logs, providing a rich source of information for training and testing your model. The data has already undergone preprocessing, and we have a target label, `sus_label`, indicating whether an event is malicious (1) or benign (0).

By successfully developing this model, you will contribute to enhancing cybersecurity measures and protecting organizations from potentially devastating cyber attacks.


### The Data

| Column     | Description              |
|------------|--------------------------|
|`processId`|The unique identifier for the process that generated the event - int64 |
|`threadId`|ID for the thread spawning the log - int64|
|`parentProcessId`|Label for the process spawning this log - int64|
|`userId`|ID of user spawning the log|Numerical - int64|
|`mountNamespace`|Mounting restrictions the process log works within - int64|
|`argsNum`|Number of arguments passed to the event - int64|
|`returnValue`|Value returned from the event log (usually 0) - int64|
|`sus_label`|Binary label as suspicous event (1 is suspicious, 0 is not) - int64|

More information on the dataset: [BETH dataset](accreditation.md)

In [73]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torchmetrics import Accuracy
from sklearn.metrics import accuracy_score  # uncomment to use sklearn

In [74]:
# Load preprocessed data
train_df = pd.read_csv('labelled_train.csv')
test_df = pd.read_csv('labelled_test.csv')
val_df = pd.read_csv('labelled_validation.csv')

# View the first 5 rows of training set
train_df.head()

,processId,threadId,parentProcessId,userId,mountNamespace,argsNum,returnValue,sus_label
0,381,7337,1,100,4026532231,5,0,1
1,381,7337,1,100,4026532231,1,0,1
2,381,7337,1,100,4026532231,0,0,1
3,7347,7347,7341,0,4026531840,2,-2,1
4,7347,7347,7341,0,4026531840,4,0,1


In [75]:
# Loading and Scaling Data
#Separate features and labels to scale the features and convert data to PyTorch tensors
X_train = train_df.drop('sus_label', axis=1).values
y_train = train_df['sus_label'].values
X_test = test_df.drop('sus_label', axis=1).values
y_test = test_df['sus_label'].values
X_val = val_df.drop('sus_label', axis=1).values
y_val = val_df['sus_label'].values

# Separating input features (X) from target labels (y) for supervised learning.


In [76]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

'''
Feature Scaling - Standardizes features to have mean=0 and std=1. This is crucial because:

Neural networks are sensitive to input scale
Prevents features with larger values from dominating
Helps with gradient descent convergence
'''

'\nFeature Scaling - Standardizes features to have mean=0 and std=1. This is crucial because:\n\nNeural networks are sensitive to input scale\nPrevents features with larger values from dominating\nHelps with gradient descent convergence\n'

In [77]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)


# Converting NumPy arrays to PyTorch tensors, which are the fundamental data structure for PyTorch operations and GPU acceleration.

In [78]:
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 128),  # Input layer → 128 neurons
    nn.ReLU(),                         # Activation function
    nn.Linear(128, 64),                # Hidden layer → 64 neurons
    nn.ReLU(),                         # Activation function
    nn.Linear(64, 1),                  # Output layer → 1 neuron
    nn.Sigmoid()                       # Final activation for binary classification
)

'''
Feedforward Neural Network: Data flows forward through layers
Hidden Layers: Two hidden layers (128 and 64 neurons) for learning complex patterns
ReLU Activation: Rectified Linear Unit (max(0,x)) - prevents vanishing gradients
Sigmoid Output: Outputs probability between 0-1 for binary classification
'''

'\nFeedforward Neural Network: Data flows forward through layers\nHidden Layers: Two hidden layers (128 and 64 neurons) for learning complex patterns\nReLU Activation: Rectified Linear Unit (max(0,x)) - prevents vanishing gradients\nSigmoid Output: Outputs probability between 0-1 for binary classification\n'

In [79]:
criterion = nn.BCELoss()  # ⚠️ Issue here - see below
optimizer = optim.Adam(model.parameters(), lr=0.001)
                       
'''
Loss Function: Measures prediction error
Optimizer: Updates model weights to minimize loss
Weight Decay: L2 regularization to prevent overfitting
Learning Rate: Controls step size in gradient descent
'''

'\nLoss Function: Measures prediction error\nOptimizer: Updates model weights to minimize loss\nWeight Decay: L2 regularization to prevent overfitting\nLearning Rate: Controls step size in gradient descent\n'

In [80]:
num_epoch = 10
for epoch in range(num_epoch):
    model.train()           # Set to training mode
    optimizer.zero_grad()   # Clear previous gradients
    outputs = model(X_train_tensor)  # Forward pass
    loss = criterion(outputs, y_train_tensor)  # Calculate loss
    loss.backward()         # Backward pass (compute gradients)
    optimizer.step()        # Update weights

'''
Epoch: One complete pass through the training data
Forward Pass: Input → Output prediction
Backward Pass: Calculate gradients using backpropagation
Gradient Descent: Update weights to minimize loss
'''

'\nEpoch: One complete pass through the training data\nForward Pass: Input → Output prediction\nBackward Pass: Calculate gradients using backpropagation\nGradient Descent: Update weights to minimize loss\n'

In [81]:
model.eval()  # Set to evaluation mode
with torch.no_grad():  # Disable gradient computation
    y_predict_train = model(X_train_tensor).round()
    y_predict_test = model(X_test_tensor).round()
    y_predict_val = model(X_val_tensor).round()

accuracy = Accuracy(task="binary")

train_accuracy = accuracy(y_predict_train, y_train_tensor)
test_accuracy = accuracy(y_predict_test, y_test_tensor)
val_accuracy = accuracy(y_predict_val, y_val_tensor)

# convert to int or float
train_accuracy = train_accuracy.item()
test_accuracy = test_accuracy.item()
val_accuracy = val_accuracy.item()

print("Training accuracy: {0}".format(train_accuracy))
print("Validation accuracy: {0}".format(val_accuracy))
print("Testing accuracy: {0}".format(test_accuracy))

Training accuracy: 0.9983371496200562
Validation accuracy: 0.9958405494689941
Testing accuracy: 0.09265109896659851


Key Issues and Improvements
The comments in the code suggest several improvements:

Use BCELoss instead of CrossEntropyLoss for binary classification
Use Adam optimizer instead of SGD for potentially better convergence
Add validation during training to monitor overfitting
Use DataLoader for batch processing (currently processes entire dataset at once)

Summary of ML Concepts Implemented

Supervised Learning: Learning from labeled examples
Binary Classification: Predicting one of two classes
Deep Learning: Multi-layer neural network
Feature Scaling: Standardizing input features
Regularization: Weight decay to prevent overfitting
Train-Validation-Test: Proper dataset splitting
Backpropagation: Gradient-based learning algorithm